Reading the csv file

In [0]:
df = spark.read.csv("/Volumes/workspace/internshipschema/internshipvolume/genAI_df.csv", header=True, inferSchema=True, multiLine=True, escape="\"", quote='\"')

In [0]:
display(df)

Diplaying the schema and counting total rows

In [0]:
df.printSchema()
display(spark.createDataFrame([(df.count(),)], ["total_rows"]))

Selecting only 4 columns and displaying top 20 rows.

In [0]:
selected_df = df.select("author", "content_cleaned", "number_of_likes", "ai_category")
display(selected_df.limit(20))

Filterings rows based on a numeric condition e.g (> average number of likes). 

In [0]:
from pyspark.sql.functions import avg

avg_likes = selected_df.agg(avg("number_of_likes")).first()[0]
filtered_df = selected_df.filter(selected_df["number_of_likes"] > avg_likes)
display(filtered_df)

Grouping by a categorical column and calculate count + average. 

In [0]:
from pyspark.sql.functions import count, avg, round

grouped_df = selected_df.groupBy("ai_category").agg(
    count("*").alias("count"),
    round(avg("number_of_likes"), 2).alias("avg_number_of_likes")
)
display(grouped_df)

Creating a new column using withColumn and a Spark SQL function. 

In [0]:
from pyspark.sql.functions import upper

new_df = selected_df.withColumn("author_upper", upper(selected_df["author"]))
display(new_df)

Handling missing values using fillna. 

In [0]:
filled_df = selected_df.fillna({"author": "Unknown", "content_cleaned": "", "number_of_likes": 0, "ai_category": "Uncategorized"})
display(filled_df.describe().show())

Writing DataFrame back to a Delta table. 

In [0]:
filled_df.write.format("delta").mode("overwrite").saveAsTable("internshipschema.databrics_processed_tweets")

Runing SQL query on created Delta table using %sql. 

In [0]:
%sql
SELECT ai_category, COUNT(*) AS count, AVG(number_of_likes) AS avg_number_of_likes
FROM internshipschema.databrics_processed_tweets
GROUP BY ai_category
ORDER BY count DESC